In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/data/image

origin.7z.001  origin.7z.003  origin.7z.005  origin.7z.007
origin.7z.002  origin.7z.004  origin.7z.006  origin.7z.008


In [ ]:
!7za -y x "/content/drive/MyDrive/data/image/origin.7z*"


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/data/image/                                            8 files, 8113576419 bytes (7738 MiB)
  0%    
Extracting archive: /content/drive/MyDrive/data/image/origin.7z.001
--
Path = /content/drive/MyDrive/data/image/origin.7z.001
Type = Split
Physical Size = 1048576000
Volumes = 8
Total Physical Size = 8113576419
----
Path = origin.7z
Size = 8113576419
--
Path = origin.7z
Type = 7z
Physical Size = 8113576419
Headers Size = 863607
Method = LZMA:25
Solid = +
Blocks = 2

  0%      0% - origin/afraid_African_214.jpg                                      0% 5

In [ ]:
!cp -r ./origin "/content/drive/MyDrive/My Projects/Expression Classification from Facial Images"

cp: cannot create directory '/content/drive/MyDrive/My Projects/Expression Classification from Facial Images': No such file or directory


In [ ]:
label_path = r"/content/drive/MyDrive/data/label/label.lst"
images_path=r"/content/drive/MyDrive/origin"

In [ ]:
import pandas as pd

face_data = pd.read_csv(label_path, sep=" " , header= None)
face_data

,0,1,2,3,4,5,6,7
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [ ]:
face_data.columns =['image_name', 'face_id_in_image', 'face_box_top', 'face_box_left' , 'face_box_right' , 'face_box_bottom' , 'face_box_cofidence' , 'expression_label']
face_data

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [ ]:
face_data.shape

(91793, 8)

In [ ]:
face_cofidence = face_data[face_data.face_box_cofidence > 55]

In [ ]:
face_cofidence.shape

(39599, 8)

In [ ]:
from numpy.core.fromnumeric import resize
import cv2
import matplotlib.pyplot as plt
import os


stored_image = []
stored_label = []

for i, row in face_cofidence.sample(10000).iterrows():
    image_name = row["image_name"]
    left = row["face_box_left"]
    right = row["face_box_right"]
    top = row["face_box_top"]
    bottom = row["face_box_bottom"]
    label = row["expression_label"]
    img_path = os.path.join(images_path, image_name)
    read_image = cv2.imread(img_path)

    if read_image is not None:
         crop_image = read_image[top:bottom , left:right]
    else:
        continue

    if crop_image  is not None:
     crop_resize = cv2.resize(crop_image  , ( 64,64))

    else:
      continue

    stored_image.append(crop_resize)
    stored_label.append(label)



In [ ]:

import numpy as np

X = np.array(stored_image)
Y = np.array(stored_label)

In [ ]:
x = X / 255.0

In [ ]:
X_shaped = x.reshape((-1,64*64*3))
X_shaped.shape

(9999, 12288)

In [ ]:
import pandas as pd
pd.Series(Y).value_counts()

,count
3,3861
6,3188
4,984
5,846
1,500
0,492
2,128


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_smote,Y_smote = smote.fit_resample(X_shaped, Y)

In [ ]:
pd.Series(Y_smote).value_counts()

,count
3,3861
4,3861
6,3861
1,3861
0,3861
5,3861
2,3861


In [ ]:
x = X_smote.reshape(-1,64,64,3)

In [ ]:
Y = Y_smote

In [ ]:
len(x)


24500

In [ ]:
len(X)

8354

In [ ]:
import numpy as np

from keras.utils import to_categorical
y = to_categorical(Y)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


img_datagen = ImageDataGenerator(

    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


img_datagen.fit(X_train)

## ***MODEL***

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
num_classes = 7
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(img_datagen.flow(X_train, Y_train, batch_size=32), epochs=50, validation_data=(X_val, Y_val))

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


592/592 ━━━━━━━━━━━━━━━━━━━━ 158s 261ms/step - accuracy: 0.1823 - loss: 1.9223 - val_accuracy: 0.2553 - val_loss: 1.8341
Epoch 2/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 199s 256ms/step - accuracy: 0.2403 - loss: 1.8488 - val_accuracy: 0.2805 - val_loss: 1.7790
Epoch 3/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 160s 270ms/step - accuracy: 0.2708 - loss: 1.8078 - val_accuracy: 0.3547 - val_loss: 1.6778
Epoch 4/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 205s 275ms/step - accuracy: 0.2946 - loss: 1.7668 - val_accuracy: 0.3485 - val_loss: 1.6527
Epoch 5/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 164s 277ms/step - accuracy: 0.3158 - loss: 1.7278 - val_accuracy: 0.3794 - val_loss: 1.6013
Epoch 6/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 203s 278ms/step - accuracy: 0.3356 - loss: 1.6851 - val_accuracy: 0.4073 - val_loss: 1.5287
Epoch 7/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 204s 282ms/step - accuracy: 0.3481 - loss: 1.6507 - val_accuracy: 0.4053 - val_loss: 1.5113
Epoch 8/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 200s 280ms/step - accuracy: 0.3605 - loss: 1.62

In [ ]:
loss ,accuracy =model.evaluate(X_test,Y_test )
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

120/120 [==============================] - 6s 51ms/step - loss: 0.9867 - accuracy: 0.6443
Test Loss: 0.9867
Test Accuracy: 0.6443
